In [1]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\YarkinErdogan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import math
import nltk
import random
import pickle
import numpy as np
import re
import pandas as pd
from sklearn.model_selection import train_test_split

In [37]:
df=pd.read_csv('train_code_completion.csv')
df.head()

,instruction,input,output,prompt
0,Create a function to calculate the sum of a se...,"[1, 2, 3, 4, 5]",# Python code\ndef sum_sequence(sequence):\n ...,Below is an instruction that describes a task....
1,Generate a Python code for crawling a website ...,website: www.example.com \ndata to crawl: phon...,import requests\nimport re\n\ndef crawl_websit...,Below is an instruction that describes a task....
2,Create a Python list comprehension to get the ...,NaN,"[x*x for x in [1, 2, 3, 5, 8, 13]]",Below is an instruction that describes a task....
3,Generate a python script to perform this action.,"Given a string, remove all the consecutive dup...",def remove_duplicates(string): \n result = ...,Below is an instruction that describes a task....
4,Write a python script to generates random numb...,NaN,def generate_random_divisible_number():\n i...,Below is an instruction that describes a task....


In [80]:
df.shape

(18612, 4)

### Preprocess Data

In [48]:
def preprocess_pipeline(data_column):
    """
    Preprocesses the text data in a pandas DataFrame column.
    
    Args:
    data_column (pd.Series): The DataFrame column containing text data.
    
    Returns:
    list: A list of tokenized sentences.
    """
    # Initialize an empty list to hold tokenized sentences
    tokenized = []
    
    # Iterate through each entry in the column
    for data in data_column:
        # Skip NaN values
        if pd.isna(data):
            continue
        
        # Split by newline character
        sentences = data.split('\n')
        
        # Remove leading and trailing spaces
        sentences = [s.strip() for s in sentences]
        
        # Drop Empty Sentences
        sentences = [s for s in sentences if len(s) > 0]
        
        # Iterate through sentences
        for sentence in sentences:
            # Convert to lowercase
            # sentence = sentence.lower()
            
            # Convert to a list of words
            token = nltk.word_tokenize(sentence)
            
            # Append to the tokenized list
            tokenized.append(token)
    
    return tokenized

# Process the 'output' column in your DataFrame
tokenized_sentences = preprocess_pipeline(df['output'])

### Train-Test Split

In [53]:
train, test = train_test_split(tokenized_sentences, test_size=0.2, random_state=42)

## Obtain Train and Validation Split 
train, val = train_test_split(train, test_size=0.25, random_state=42)

### Word Counts

In [50]:
def count_the_words(sentences) -> 'dict':
    word_counts = {}

  # Iterating over sentences
    for sentence in sentences:
    
    # Iterating over Tokens
        for token in sentence:
    
      # Add count for new word
            if token not in word_counts.keys():
                word_counts[token] = 1
        
      # Increase count by one
            else:
                word_counts[token] += 1
        
    return word_counts

### Handling Out-of-Vocabulary Error

In [51]:
def handling_oov(tokenized_sentences, count_threshold) -> 'list':
    closed_vocabulary = []

  # Obtain frequency dictionary using previously defined function
    words_count = count_the_words(tokenized_sentences)
    
  # Iterate over words and counts 
    for word, count in words_count.items():
    
    # Append if it's more(or equal) to the threshold 
        if count >= count_threshold :
            closed_vocabulary.append(word)

    return closed_vocabulary

### Tokenization of Unknown Words

In [54]:
def unk_tokenize(tokenized_sentences, vocabulary, unknown_token = "<unk>") -> 'list':
    vocabulary = set(vocabulary)

  # Create empty list for sentences
    new_tokenized_sentences = []
  
  # Iterate over sentences
    for sentence in tokenized_sentences:

    # Iterate over sentence and add <unk> 
    # if the token is absent from the vocabulary
        new_sentence = []
        for token in sentence:
            if token in vocabulary:
                new_sentence.append(token)
            else:
                new_sentence.append(unknown_token)
    
    # Append sentece to the new list
        new_tokenized_sentences.append(new_sentence)

    return new_tokenized_sentences

### OOV Rate Comparison between Baseline and Main Approach

In [112]:
# Example tokenized sentences and vocabulary
def simple_tokenize(text):
    return re.findall(r'\w+|[^\w\s]', text)


tokenized_sentences = [["for", "x", "in", "range(10)"], ["def", "foo", "()", ":"]]
vocabulary = ["for", "x", "in", "range", "def", "foo", "10"]


# Apply unk_tokenize
modified_sentences = unk_tokenize(tokenized_sentences, vocabulary)

# Apply baseline simple tokenization (Whitespace tokenization)
baseline_sentences = [simple_tokenize(" ".join(sentence)) for sentence in tokenized_sentences]


# Compare the number of OOV tokens in both approaches
def oov_rate(tokenized_sentences, vocabulary):
    total_tokens = sum(len(sentence) for sentence in tokenized_sentences)
    unk_tokens = sum(1 for sentence in tokenized_sentences for token in sentence if token not in vocabulary)
    return unk_tokens / total_tokens

print("OOV rate for your method:", oov_rate(modified_sentences, vocabulary))
print("OOV rate for the baseline:", oov_rate(baseline_sentences, vocabulary))


OOV rate for your method: 0.375
OOV rate for the baseline: 0.4166666666666667


### Cleaning the Data

In [55]:
def cleansing(train_data, test_data, count_threshold):
    vocabulary = handling_oov(train_data, count_threshold)
    
  # Updated Training Dataset
    new_train_data = unk_tokenize(train_data, vocabulary)
    
  # Updated Test Dataset
    new_test_data = unk_tokenize(test_data, vocabulary)

    return new_train_data, new_test_data, vocabulary

In [56]:
min_freq = 6
final_train, final_test, vocabulary = cleansing(train, test, min_freq)

### Count N-grams

In [57]:
def count_n_grams(data, n, start_token = "<s>", end_token = "<e>") -> 'dict':
    n_grams = {}
 
  # Iterate over all sentences in the dataset
    for sentence in data:
        sentence = [start_token]*n + sentence + [end_token]
    
    # Convert the sentence into a tuple
        sentence = tuple(sentence)

    # Temp var to store length from start of n-gram to end
        m = len(sentence) if n==1 else len(sentence)-1
    
    # Iterate over this length
        for i in range(m):
        
      # Get the n-gram
            n_gram = sentence[i:i+n]
    
      # Add the count of n-gram as value to our dictionary
      # IF n-gram is already present
            if n_gram in n_grams.keys():
                n_grams[n_gram] += 1
      # Add n-gram count
            else:
                n_grams[n_gram] = 1
        
    return n_grams

### Probability for Single Word

In [58]:
def prob_for_single_word(word, previous_n_gram, n_gram_counts, nplus1_gram_counts, vocabulary_size, k = 1.0) -> 'float':
    previous_n_gram = tuple(previous_n_gram)
    
  # Calculating the count, if exists from our freq dictionary otherwise zero
    previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0
  
  # The Denominator
    denom = previous_n_gram_count + k * vocabulary_size

  # previous n-gram plus the current word as a tuple
    nplus1_gram = previous_n_gram + (word,)

  # Calculating the nplus1 count, if exists from our freq dictionary otherwise zero 
    nplus1_gram_count = nplus1_gram_counts[nplus1_gram] if nplus1_gram in nplus1_gram_counts else 0

  # Numerator
    num = nplus1_gram_count + k

  # Final Fraction
    prob = num / denom
    return prob

### Proability Calculation

In [59]:
def probs(previous_n_gram, n_gram_counts, nplus1_gram_counts, vocabulary, k=1.0) -> 'dict':
    previous_n_gram = tuple(previous_n_gram)

  # Add end and unknown tokens to the vocabulary
    vocabulary = vocabulary + ["<e>", "<unk>"]

  # Calculate the size of the vocabulary
    vocabulary_size = len(vocabulary)

  # Empty dict for probabilites
    probabilities = {}

  # Iterate over words 
    for word in vocabulary:
    
    # Calculate probability
        probability = prob_for_single_word(word, previous_n_gram, 
                                           n_gram_counts, nplus1_gram_counts, 
                                           vocabulary_size, k=k)
    # Create mapping: word -> probability
        probabilities[word] = probability

    return probabilities

### Auto Completion

In [60]:
def auto_complete(previous_tokens, n_gram_counts, nplus1_gram_counts, vocabulary, k=1.0, start_with=None):

    
    # length of previous words
    n = len(list(n_gram_counts.keys())[0]) 
    
    # most recent 'n' words
    previous_n_gram = previous_tokens[-n:]
    
    # Calculate probabilty for all words
    probabilities = probs(previous_n_gram,n_gram_counts, nplus1_gram_counts,vocabulary, k=k)

    # Intialize the suggestion and max probability
    suggestion = None
    max_prob = 0

    # Iterate over all words and probabilites, returning the max.
    # We also add a check if the start_with parameter is provided
    for word, prob in probabilities.items():
        
        if start_with != None: 
            
            if not word.startswith(start_with):
                continue 

        if prob > max_prob: 

            suggestion = word
            max_prob = prob

    return suggestion, max_prob

### Get Suggestions

In [61]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):

    # See how many models we have
    count = len(n_gram_counts_list)
    
    # Empty list for suggestions
    suggestions = []
    
    # IMP: Earlier "-1"
    
    # Loop over counts
    for i in range(count-1):
        
        # get n and nplus1 counts
        n_gram_counts = n_gram_counts_list[i]
        nplus1_gram_counts = n_gram_counts_list[i+1]
        
        # get suggestions 
        suggestion = auto_complete(previous_tokens, n_gram_counts,
                                    nplus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with)
        # Append to list
        suggestions.append(suggestion)
        
    return suggestions

In [62]:
n_gram_counts_list = []
for n in range(1, 6):
    n_model_counts = count_n_grams(final_train, n)
    n_gram_counts_list.append(n_model_counts)

### Trials

In [169]:
previous_tokens = previous_tokens = ["math", ".","sqrt", "("]
suggestion = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

display(suggestion)

[('<unk>', 0.13515289360626737),
 ('x', 0.3181818181818182),
 ('for', 0.1111111111111111),
 ('for', 0.1111111111111111)]

In [180]:
previous_tokens = previous_tokens = ["[", "x*x", "for","x","in"]
suggestion = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

display(suggestion)

[('range', 0.30852360445275046),
 ('range', 0.3599137931034483),
 ('range', 0.3874709976798144),
 ('for', 0.1111111111111111)]

### Top-1 Accuracy

In [178]:
def top_1_accuracy(previous_tokens, n_gram_counts_list, vocabulary, true_token=None):
    # Get the top suggestion from the model (k=1)
    suggestions = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)
    
    # Extract the top 1 suggestion
    top_1_token = suggestions[0][0]
    
    # Check if the true token is the top 1 suggestion
    if top_1_token == true_token:
        return 1
    else:
        return 0

# Example usage

previous_tokens = ["math", ".","sqrt", "("]
true_token = "x" # the correct next token

accuracy_top_1 = top_1_accuracy(previous_tokens, n_gram_counts_list, vocabulary, true_token=true_token)
print(f"Top-1 Accuracy: {accuracy_top_1}")


Top-1 Accuracy: 0


### Top-k Accuracy

In [179]:
def top_k_accuracy(previous_tokens, n_gram_counts_list, vocabulary, k=1, true_token=None):
    # Get the top-k suggestions from the model
    suggestions = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)
    
    # Flatten suggestions to get only the tokens (ignoring the probabilities)
    top_k_tokens = [s[0] for s in suggestions][:k]
    
    # Check if the true token is in the top-k suggestions
    if true_token in top_k_tokens:
        return 1
    else:
        return 0

# Example usage
previous_tokens = ["math", ".","sqrt", "("]
true_token = "x" # the correct next token
top_k = 4  # specify the value of k

# Assuming n_gram_counts_list and vocabulary are defined previously
accuracy = top_k_accuracy(previous_tokens, n_gram_counts_list, vocabulary, k=top_k, true_token=true_token)
print(f"Top-{top_k} Accuracy: {accuracy}")


Top-4 Accuracy: 1


### MRR

In [177]:
def mrr(previous_tokens, n_gram_counts_list, vocabulary, true_token=None):
    # Get the top-k suggestions from the model (we assume k=all for MRR)
    suggestions = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1000)
    
    # Find the rank of the true token (it’s the first occurrence in the list)
    for rank, (word, _) in enumerate(suggestions, 1):
        if word == true_token:
            return 1 / rank  # Reciprocal rank

    # If the true token is not found, return 0
    return 0

# Example usage
previous_tokens = ["math", ".","sqrt", "("]
true_token = "x" # the correct next token

mrr_value = mrr(previous_tokens, n_gram_counts_list, vocabulary, true_token=true_token)
print(f"MRR: {mrr_value}")

MRR: 0.5
